In [18]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
BRANCH = 'r1.4.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml
!pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.4.0) to /tmp/pip-install-mlch1wvx/nemo-toolkit_6e41ed7d81d541cd80a6b20d0ed9d999
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-mlch1wvx/nemo-toolkit_6e41ed7d81d541cd80a6b20d0ed9d999
  Running command git checkout -b r1.4.0 --track origin/r1.4.0
  Switched to a new branch 'r1.4.0'
  Branch 'r1.4.0' set up to track remote branch 'r1.4.0' from 'origin'.
  Using cached matplotlib-3.5.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
  Attempting uninstall: matplotlib
    Found existing

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

In [19]:
# Папка, где будет размещаться датасет Golos
data_dir = '/content'

In [ ]:
import glob
import os
import subprocess
import tarfile
import wget

# Загрузка датасета Golos
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

golos_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(golos_path)
    tar.extractall(path=data_dir)

In [ ]:
! head /content/test/crowd/manifest.jsonl

In [ ]:
import shutil

shutil.rmtree('/content/sample_data')

Теперь в папке `/content` должна размещаться папка `test` с тестовыми данных датасета Golos. Она содержит файлы манифесты `./test/crowd/manifest.jsonl`, `./test/farfield/manifest.jsonl` и аудио файлы в подпапках `./test/crowd/files/` и `./test/farfield/files/`.

In [ ]:
golos_path_train = load('https://sc.link/XKk', 'train_crowd9.tar')

if not os.path.exists(os.path.join(data_dir, '/train/')):
    tar = tarfile.open(golos_path_train)
    tar.extractall(path=data_dir)

In [ ]:
shutil.move('/content/train/manifest.jsonl', '/content')

In [ ]:
shutil.rmtree('/content/train/')

In [ ]:
golos_path_train = load('https://sc.link/1Z3', 'train_farfield.tar')

if not os.path.exists(os.path.join(data_dir, '/train/')):
    tar = tarfile.open(golos_path_train)
    tar.extractall(path=data_dir)

## Сверточная модель распознавания речи
### Jasper модель

Мы будем тренировать небольшую модель [Jasper (Just Another SPeech Recognizer) model](https://arxiv.org/abs/1904.03288) с нуля (инициализируется случайным образом).
Архитектуры Jasper состоят из повторяющейся блочной структуры, которая использует 1D-свертки.
В модели Jasper_KxR подблоки `R` (состоящие из 1D свертки, batchnorm, ReLU и dropout) группируются в один блок, который затем повторяется `K` раз.
Также есть один дополнительный блок в начале и еще несколько в конце, которые инвариантны к `K` и `R`, идалее используем CTC loss.

### Модель QuartzNet

QuartzNet является улучшенным вариантом Jasper с ключевым отличием в том, что она использует 1D-свертки, разделяемые по временному каналу. Это позволяет ему значительно сократить количество весов при сохранении аналогичной точности.

Модели Jasper/QuartzNet выглядят так (на фото изображена модель QuartzNet):

![QuartzNet with CTC](https://developer.nvidia.com/blog/wp-content/uploads/2020/05/quartznet-model-architecture-1-625x742.png)

# Использование NeMo для распонавания речи

Теперь мы знаем что такое задача автоматичского распознавания речи и речевые данные, давайте использовать NeMo для распознавания речи.

Мы будем использовать **Neural Modules (NeMo) toolkit**, поэтому нужно скачать и установить все ее зависимости. Для этого следуйте инструкции в репозитории [GitHub page](https://github.com/NVIDIA/NeMo), или документации [documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/).

NeMo позволяет нам легко использовать все необходимые компоненты для нашей модели: dataloader,  промежуточные сверточные или рекурентные слои, разные loss  функции без необходимости разбираться в деталях реализации разных моедлей. В NeMo содержатся готовые реализованные модели в которых достаточно подать свои данные и задать гиперпараметры для обучения.

In [ ]:
import shutil

shutil.rmtree('/content/test/crowd')

In [ ]:
!pip install torch==1.10.0
!pip install pytorch-lightning
!pip install torchaudio==0.9.1
!pip install torchtext==0.10.1

In [ ]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

## Исползование предобученной модели


Коллекция для распознавания речи в NeMo содержит готовые блоки, которые можно использовать чтобы тренировать и использовать свою модель. Кроме этого существет ряд предобученных моделей, которые можно просто скачать и исползовать. Давайте скачаем и инициализируем готовую модель QuartzNet15x5, обученную на открытом датасете Golos.



In [ ]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")

asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

In [ ]:
!pip install jiwer

     |████████████████████████████████| 50 kB 5.5 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149871 sha256=f9a236d16af1737a339dc089e48d6b718a020986f479b692232ef8b62bd7f271
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
from jiwer import wer

Теперь указваем список фалов которые мы хотим транскрибировать и передаем в нашу модель. Это будет работать для относительно коротких аудио (<25 секунд) файлов. 

In [ ]:
import json
 
with open('/content/test/farfield/manifest.jsonl') as f:
    manifest_test = [json.loads(line) for line in f]

In [21]:
files = ["/content/test/farfield/" + file['audio_filepath'] for file in manifest_test]
transcriptions = asr_model.transcribe(paths2audio_files=files)

Transcribing:   0%|          | 0/479 [00:00<?, ?it/s]

In [34]:
wers = []
info = []

for ind, transcription in enumerate(transcriptions):
  if ind == 1077:
    continue
  error = wer(manifest_test[ind]['text'], transcription)
  wers.append(error)
  if error > .30:
    info.append({})
    print('\tIND:', ind)
    info[-1]['id'] = ind
    print('WER Error: ', error)
    info[-1]['wer'] = error
    print('Ground truth: ', manifest_test[ind]['text'])
    info[-1]['duration'] = manifest_test[ind]['duration']
    info[-1]['length'] = len(manifest_test[ind]['text'].split())
    print('Predicted: ', transcription)

	IND: 2
WER Error:  1.0
Ground truth:  салют хватит
Predicted:  али лат
	IND: 10
WER Error:  0.5
Ground truth:  сбер громче
Predicted:  сбер грамче
	IND: 18
WER Error:  0.5
Ground truth:  джой выйти
Predicted:  джой выдте
	IND: 21
WER Error:  0.5
Ground truth:  джой выйди
Predicted:  джой выди
	IND: 22
WER Error:  0.3333333333333333
Ground truth:  сбер набери изотова
Predicted:  сбер набери изотово
	IND: 24
WER Error:  0.3333333333333333
Ground truth:  афина набери марию
Predicted:  афина набери мария
	IND: 32
WER Error:  1.0
Ground truth:  салют приложение сверни
Predicted:  али
	IND: 34
WER Error:  0.5
Ground truth:  сбер надоело
Predicted:  сбер надоела
	IND: 57
WER Error:  0.5
Ground truth:  афина надоела
Predicted:  афина надоело
	IND: 62
WER Error:  0.5
Ground truth:  афина вызов владлену леонидовичу
Predicted:  афина вызов владлена леонидовича
	IND: 67
WER Error:  0.3333333333333333
Ground truth:  салют набери юнусова
Predicted:  салют набери юнусовао
	IND: 78
WER Error:  0.5
Gr

In [35]:
print('Len WERs: ', len(wers))
print('Mean WER: ', sum(wers)/len(wers))

Len WERs:  1915
Mean WER:  0.15346754638550314


In [36]:
info

[{'duration': 1.5226875, 'id': 2, 'length': 2, 'wer': 1.0},
 {'duration': 1.54925, 'id': 10, 'length': 2, 'wer': 0.5},
 {'duration': 1.3928125, 'id': 18, 'length': 2, 'wer': 0.5},
 {'duration': 1.3419375, 'id': 21, 'length': 2, 'wer': 0.5},
 {'duration': 2.012, 'id': 22, 'length': 3, 'wer': 0.3333333333333333},
 {'duration': 1.7095625, 'id': 24, 'length': 3, 'wer': 0.3333333333333333},
 {'duration': 1.9996875, 'id': 32, 'length': 3, 'wer': 1.0},
 {'duration': 2.009625, 'id': 34, 'length': 2, 'wer': 0.5},
 {'duration': 2.04775, 'id': 57, 'length': 2, 'wer': 0.5},
 {'duration': 3.83075, 'id': 62, 'length': 4, 'wer': 0.5},
 {'duration': 1.6818125, 'id': 67, 'length': 3, 'wer': 0.3333333333333333},
 {'duration': 1.3445625, 'id': 78, 'length': 2, 'wer': 0.5},
 {'duration': 1.13175, 'id': 82, 'length': 2, 'wer': 0.5},
 {'duration': 1.80075, 'id': 83, 'length': 3, 'wer': 0.3333333333333333},
 {'duration': 3.27775, 'id': 86, 'length': 5, 'wer': 0.6},
 {'duration': 2.8555625, 'id': 87, 'length'

In [37]:
len(info)

448

In [45]:
durations = [inf['duration'] for inf in info]
lens = [inf['length'] for inf in info]

print(sum(durations)/len(durations))
print(sum(lens)/len(lens))

print(sorted([{length: lens.count(length) for length in set(lens)}], reverse=True))

2.532504603794642
3.9464285714285716
[{1: 1, 2: 99, 3: 147, 4: 74, 5: 50, 6: 39, 7: 9, 8: 8, 9: 12, 10: 1, 11: 2, 12: 4, 14: 1, 19: 1}]


In [58]:
ids_problem = []
index = 0
inds = -1

with open('/content/train/train_manifest.jsonl') as f:
  for line in f:
    data = json.loads(line)
    if data['duration'] < 4 and len(data['text'].split()) in list(range(2, 7)):
      ids_problem.append({index: data['id']})
    index += 1

print(len(ids_problem))

with open('/content/train/train_manifest.jsonl') as f:
  for line in f:
    inds += 1
  
print(inds)

57541
124002


Это было просто! Но существует масса сценариев, когда мы хотим дообучить модель на наших данных или обучить заново. Например это модель не сможет распознать украинский язык и скоре всего будет плохо распознавать речь в телефонных разговорах. Поэтому, если вы применяете модель на своих данных, то без сомнения, вам нужно дообучать модель на них!

## Обучение с нуля


Для обучения нужно подготовить данные в нужном формате. Для этого добавим абсолютные пути к относительным в наших манифестах и используем их для обучения.

In [40]:
shutil.move('/content/manifest.jsonl', '/content/train/manifest.jsonl')

'/content/train/manifest.jsonl'

In [92]:
# --- Building Manifest Files --- #
import json

idificators = tuple([list(dictat.values())[0] for dictat in ids_problem])
id_go_through = 0

# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs):
    global id_go_through
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
              metadata = json.loads(line)
              if 'train' in manifest_rel and metadata['id'] in idificators and 'farfield' in metadata["audio_filepath"]:  # metadata['id'] in idificators  and 'farfield' in metadata["audio_filepath"]
                  id_go_through += 1
                  metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                  json.dump(metadata, fout)
                  fout.write('\n')
              elif 'test' in manifest_rel:
                  metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                  json.dump(metadata, fout)
                  fout.write('\n')
                
# Building Manifests
print("******")
test_rel = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
test_abs = os.path.join(data_dir, 'test/farfield/farfield.jsonl')
if not os.path.isfile(test_abs):
  build_manifest(test_rel, test_abs)
test_manifest = test_abs
print("test_manifest", test_manifest)

train_rel = os.path.join(data_dir, 'train/manifest.jsonl')
train_abs = os.path.join(data_dir, 'train/train_manifest.jsonl')
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs)
train_manifest = train_abs
print("train_manifest", train_manifest)

******
test_manifest /content/test/farfield/farfield.jsonl
train_manifest /content/train/train_manifest.jsonl


In [93]:
id_go_through

57541

In [94]:
with open('/content/train/train_manifest.jsonl') as f:
  for line in f:
    data = json.loads(line)
    if 'crowd' in data['audio_filepath']:
      print(data)

In [43]:
# ! head /content/test/farfield/manifest.jsonl
! head /content/train/train_manifest.jsonl

{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "/content/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0445\u043e\u0442\u0435\u043b\u0430 \u043d\u043e\u043c\u0435\u0440 \u0442\u0435\u043b\u0435\u0444\u043e\u043d\u0430 \u043f\u043e\u043c\u0435\u043d\u044f\u0442\u044c \u043a\u0430\u0440\u0442\u044b", "duration": 5.4596875}
{"id": "f6075fae0c592183f107cf781117d9b5", "audio_filepath": "/content/train/farfield/f6075fae0c592183f107cf781117d9b5.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u043e\u0447\u0443 \u043f\u043e\u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0441 \u0434\u0436\u043e\u0439", "duration": 4.15125}
{"id": "2599a6cb55795b38ddd80b849d8670bc", "audio_filepath": "/content/train/farfield/2599a6cb55795b38ddd80b849d8670bc.wav", "text": "\u0434\u0436\u043e\u0439 \u043a\u0430\u043a\u0430\u044f \u0432\u0430\u043b\u044e\u0442\u0430 \u0432 \u043a\u0430\u043c\u0431\u043e\u0434\u0436\u0435", "duration": 3.9185625}

### Задаем модель при помощи YAML конфиг файла

Для обучения мы создадим модель *Jasper_4x1*, в которой будет `K=4` блоков, один (`R=1`) под-блок и декодер *greedy CTC*, используя конфиг файл в `./configs/config.yaml`.


Ниже приведен конфиг файл, давайте рассмотрим его и найдем части описанной архитектуры Jasper. Модель (model) содержит поле под названием `encoder` с под-полем `jasper` который состоит из списка полей. Каждое поле в списке задает конфигурацию блока в нашей моделе. Каждый блок выглядит примерно так:

```
- filters: 128
  repeat: 1
  kernel: [11]
  stride: [2]
  dilation: [1]
  dropout: 0.2
  residual: false
  separable: true
  se: true
  se_context_size: -1
```


Первый элемент в спике соответствует первому блоку в Jasper архитектуре.

Параметры обучающего и тестового датасета в полях (`train_ds`) и (`validation_ds`)

Конфиг в формате YAML позволяем легко и в читаемой форме читать и модифицировать модель без необходимости менять код пррограммы.

### Использование PyTorch Lightning

NeMo модели и модули могуть исползоваться в любом PyTorch проекте где ожидается тип torch.nn.Module.

Однако, NeMo модели созданы на основе [PytorchLightning's](https://github.com/PyTorchLightning/pytorch-lightning) LightningModule, поэтому рекомендуется использовать PytorchLightning для обучения и дообучения (fine-tuning) так как это позволяет легко применять mixed precision и распределенное обучение. Давайте создадим объект Trainer для обучения на GPU 5 эпох.

In [64]:
import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=1)  # gpus=0, max_epochs=1, precision=16

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [65]:
params_model = asr_model._cfg

# --- Config Information ---#
# try:
#     from ruamel.yaml import YAML
# except ModuleNotFoundError:
#     from ruamel_yaml import YAML
# config_path = '/usr/local/lib/python3.7/dist-packages/nemo/collections/asr/models/configs/'

# yaml = YAML(typ='safe')
# with open(config_path) as f:
#     params = yaml.load(f)
# print(params)
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16

In [66]:
params_model

{'sample_rate': 16000, 'repeat': 5, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'optim': {'name': 'novograd', 'lr': 0.05, 'betas': [0.9, 0.98], 'weight_decay': 0.001, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': 500, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}, 'train_ds': {'manifest_filepath': '/content/train/train_manifest.jsonl', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'num_workers': 20, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'tarred_shard_strategy': 'scatter', 'parser': 'base'}, 'validation_ds': {'manifest_filepath': '/content/te

In [67]:
params_model['optim']['lr'] = 0.001

In [68]:
from omegaconf import DictConfig


first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params_model), trainer=trainer)

[NeMo I 2021-11-30 18:12:08 collections:173] Dataset loaded with 123995 files totalling 132.43 hours
[NeMo I 2021-11-30 18:12:08 collections:174] 8 files were filtered totalling 0.03 hours


[NeMo W 2021-11-30 18:12:08 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 18:12:08 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 18:12:08 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 18:12:08 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 18:12:08 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 18:12:08 features:262] PADDING: 16
[NeMo I 2021-11-30 18:12:08 features:279] STFT using torch


In [ ]:
# model.freeze()

In [ ]:
# Start training!!!
trainer.fit(first_asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2021-11-30 13:49:59 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2021-11-30 13:49:59 lr_scheduler:625] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f62c3dcea10>" 
    will be used during training (effective maximum steps = 7750) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 7750
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.718    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 13:49:59 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


Training: -1it [00:00, ?it/s]

[NeMo W 2021-11-30 13:50:08 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
      "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
    


In [ ]:
first_asr_model.save_to('/content')

In [ ]:
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

Below is an example of a simple inference loop in pure PyTorch. It also shows how one can compute Word Error Rate (WER) metric between predictions and references.

In [ ]:
# Bigger batch-size = bigger throughput
params['model']['validation_ds']['batch_size'] = 16

# Setup the test data loader and make sure the model is on GPU
first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
first_asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

**Results**: 

Так как Colab ограничил мне доступ к GPU, я не смогу закончить это задание в течение следующих суток, поэтому отпарвлю такие результаты и опишу свой предполагаемый план действий.

1. Во-первых, я протестировала готовую загруженную модель и получила следующий WER (см. в клетке ниже)

2. Я оценила тестовые файлы, на которых WER > 0.30, а конкретно их среднюю длительность и среднюю длинну слов, а также распределение длин слов по частотности.

3. Выделила среди трейновых файлов те, которые имеют длительность меньше 4 секунд и количество слов от 2 до 6 (получилось 57541 файл из 124002).

*4. Далее я бы обучила модель на этих файлах и посмотрела результаты тестов, но сейчас я это сделать, к сожалению, не могу.

In [95]:
print('Mean WER: ', sum(wers)/len(wers))

Mean WER:  0.15346754638550314
